In [1]:
import numpy as np
import pandas as pd

try:
    from raagam_player import RaagPlayer
except:
    import pyaudio
    import numpy as np
    import wave

    class RaagPlayer:
        CHUNK = 1024
        FORMAT = pyaudio.paInt16
        CHANNELS = 2
        RATE = 44100 
        DURATION = 1.5
        base = 200

        notes_list = {'S':1, 
                        'R1':16/15,
                        'R2':9/8,
                        'R3':1.186,
                        'G1':6/5,
                        'G2':81/64,
                        'G3':81/64 + 1e-2,
                        'M1':4/3,
                        'M2':45/32,
                        'P':3/2,
                        'D1':8/5,
                        'D2':27/16,
                        'D3':1.69,
                        'N1':16/9,
                        'N2':15/8,
                        'N3':1.9
        }
        
        def __init__(self, play_notes):
            print(play_notes)
            self.generate_note(self.aaro_avro(play_notes))


        def aaro_avro(self, raag):
            freq_tuple = [(n,self.notes_list[n]) for n in raag]
            return [(note,self.base*v) for note,v in freq_tuple + [('S2',2),('\n',0),('\rS2',2)] + freq_tuple[::-1]]

        def generate_note(self, notes):
            
            p = pyaudio.PyAudio()
            stream = p.open(format=self.FORMAT,
                            channels=self.CHANNELS,
                            rate=self.RATE,
                            output=True)
            
            for (n,freq) in notes:
                if n in ['S','P']: n+=' '
                print(n, end=" ")
                samples = [int(np.sin(2*np.pi*n*freq/self.RATE) * 32767) for n in range(int(self.RATE*self.DURATION))]
                packed_samples = wave.struct.pack('h'*len(samples), *samples)  # 'h' for signed short
                stream.write(packed_samples)
                stream.stop_stream()
                stream.close()

            p.terminate()



try: from ipywidgets import Dropdown
except: 
    !pip install ipywidgets
    from ipywidgets import Dropdown


In [2]:
mela_df = pd.read_excel(r"Melakarta Raagams.xlsx")
mela_df.columns = ["_".join(i.strip().split()) for i in mela_df.columns]

for n in ['AROHANAM','AVAROHANA']:
    mela_df[n]  = mela_df[n].apply(lambda x: x.replace("\n","").split())
mela_df.head()

,MELA_NUMBER,MELA_NAME,AROHANAM,AVAROHANA
0,1,Kanakangi,"[S, R1, G1, M1, P, D1, N1, S]","[S, N1, D1, P, M1, G1, R1, S]"
1,2,Ratnangi,"[S, R1, G1, M1, P, D1, N2, S]","[S, N2, D1, P, M1, G1, R1, S]"
2,3,Ganamurti,"[S, R1, G1, M1, P, D1, N3, S]","[S, N3, D1, P, M1, G1, R1, S]"
3,4,Vanaspati,"[S, R1, G1, M1, P, D2, N2, S]","[S, N2, D2, P, M1, G1, R1, S]"
4,5,Manavati,"[S, R1, G1, M1, P, D2, N3, S]","[S, N3, D2, P, M1, G1, R1, S]"


In [3]:
raag_i = np.random.choice(range(mela_df.shape[0]),size=1)[0]
raag = mela_df.iloc[raag_i].to_dict()
print(raag['MELA_NAME'])
RaagPlayer(list(raag['AROHANAM'])[:-1])


Natabhairavi
['S', 'R2', 'G2', 'M1', 'P', 'D1', 'N2']
S  R2 G2 M1 P  D1 N2 S2 
S2 N2 D1 P  M1 G2 R2 S  

In [4]:
d = Dropdown(options = sorted(mela_df['MELA_NAME']))

def print_selected(change):
  selected_value = change['new']  
  print(f"You selected: {selected_value}")
  RaagPlayer(mela_df[mela_df['MELA_NAME']==selected_value]['AROHANAM'].tolist()[0][:-1])
  print("\n")
d.observe(print_selected, names='value')
display(d)

Dropdown(options=('Bhavapriya', 'Chakravakam', 'Chalanata', 'Charukesi', 'Chitrambari', 'Dharmavati', 'Dhatuva…

You selected: Chitrambari
['S', 'R2', 'G3', 'M2', 'P', 'D3', 'N3']
S  R2 G3 M2 P  D3 N3 S2 
S2 N3 D3 P  M2 G3 R2 S  

You selected: Dheerashankarabharanam
['S', 'R2', 'G3', 'M1', 'P', 'D2', 'N3']
S  R2 G3 M1 P  D2 N3 S2 
S2 N3 D2 P  M1 G3 R2 S  

You selected: Gayakapriya
['S', 'R1', 'G3', 'M1', 'P', 'D1', 'N1']
S  R1 G3 M1 P  D1 N1 S2 
S2 N1 D1 P  M1 G3 R1 S  

You selected: Harikambhoji
['S', 'R2', 'G3', 'M1', 'P', 'D2', 'N2']
S  R2 G3 M1 P  D2 N2 S2 
S2 N2 D2 P  M1 G3 R2 S  

You selected: Dheerashankarabharanam
['S', 'R2', 'G3', 'M1', 'P', 'D2', 'N3']
S  R2 G3 M1 P  D2 N3 S2 
S2 N3 D2 P  M1 G3 R2 S  

You selected: Dhavalambari
['S', 'R1', 'G3', 'M2', 'P', 'D1', 'N1']
S  R1 G3 M2 P  D1 N1 S2 
S2 N1 D1 P  M2 G3 R1 S  



In [5]:
len(mela_df['MELA_NAME'])

72